# Ch10. 전처리 실습 - San Francisco Crime
---
* 날짜:2022-04-14
* 이름:안창덕

**소개**

  - [산프란시스코 범죄 데이터셋](https://www.kaggle.com/c/sf-crime) 전처리를 진행 합니다.
  - [파이썬 데이터전처리 실습-San francisco crime](https://wikidocs.net/86627)를 참고하여 전처리를 진행 합니다.
  - 향후 이 코드를 이어서 사용하니 코드를 잘 정리해 주세요.


## 데이터 전처리
---

In [ ]:
import pandas as pd
import numpy as np

**01. csv 파일을 데이터프레임으로 읽기**

In [ ]:
# zip 파일 압축 풀기
!unzip '/content/sampleSubmission.csv.zip'
!unzip '/content/test.csv.zip'
!unzip '/content/train.csv.zip'
# csv 파일을 데이터프레임으로 읽기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sampleSubmission.csv')

Archive:  /content/sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    
Archive:  /content/test.csv.zip
  inflating: test.csv                
Archive:  /content/train.csv.zip
  inflating: train.csv               


**02. 학습 데이터 셋의 상위 5개 항목 출력**

In [ ]:
train.head() #train데이터의 처음행부터 5행까지의 데이터 전부 출력

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


**03. 학습 데이터 셋의 각 필드 데이터 분포(평균, 최소값, 최대값 등)를 살펴보기**

In [ ]:
train.describe() #describe 함수를 통해 데이터의 기술통계량을 출력

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


**04. 각 컬럼의 데이터 타입 확인하기**

**※**info함수를 통해 데이터의 개략적인 정보를 확인할 수 있다.

In [ ]:
train.info() #info 함수를 통해 dtpye 정보 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


**05. 범죄의 범주 'Category'는 어떤 값이 있는지 확인하기**

**※**unique 함수는 데이터의 모든 값을 중복제거하여 출력한다. 즉 데이터 내의 단일값만을 살펴볼 수 있다.

In [ ]:
train['Category'].unique() #Category열의 단일값 출력

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

**06. Dates 필드의 데이터타입을 'datetime64' 타입으로 변경하기**

astype함수는 데이터 타입을 변경시켜준다.

딕셔너리를 활용해 특정 열의 데이터 타입만 변경하는 것도 가능하다. 예) train.astype({'Dates':'datetime64'})

In [ ]:
train['Dates'] = train['Dates'].astype('datetime64') #Dates열의 데이터 타입을 'datetime64'로 변경

**07. 연, 월, 일, 요일, 시, 분값을 새로운 컬럼(year, month, day, dayofweek, hour, minute)을 생성해서 할당하기**

**※**dt함수는 해당 데이터의 데이터타입이 'datetime64'일 때 연,월,일,요일,시,분 값을 추출할 수 있다. 


In [ ]:
train['year'] = train['Dates'].dt.year #Dates열에서 연을 추출하여 year열을 추가
train['month'] = train['Dates'].dt.month #Dates월에서 연을 추출하여 month열을 추가
train['day'] = train['Dates'].dt.day #Dates열에서 일을 추출하여 day열을 추가
train['dayofweek'] = train['Dates'].dt.dayofweek #Dates열을 날짜를 통해 요일을 추출하여 dayofweek열에 추가
train['hour'] = train['Dates'].dt.hour #Dates열에서 시(hour)을 추출하여 hour열을 추가
train['minute'] = train['Dates'].dt.minute #Dates열에서 분(minute)을 추출하여 minute열을 추가

**08. 범죄가 발생한 경도, 위도 값을 나타내는 X, Y 컬럼의 값을 더한 값을 담은 'X+Y' 컬럼, X, Y의 차이 값을 담은 'X-Y' 컬럼을 생성하기**


In [ ]:
train['X+Y'] = train['X'] + train['Y'] # X열과 Y열을 합친 'X+Y'열을 추가
train['X-Y'] = train['X'] - train['Y'] # X열에 Y을 뺀 'X-Y'열을 추가

**09. n_days라는 컬럼을 생성하고, 일별 d-day를 계산하여 입력하기**


In [ ]:
train.info()

In [ ]:
test['Dates'] = test['Dates'].astype('datetime64')

In [ ]:
train['n_days'] = (train['Dates'].dt.date - train['Dates'].dt.date.min()).apply(lambda x: x.days) #train데이터 프레임의 tates열에서 날짜값에 가장 작은 날짜값의 차이를 구해서 x.days형태로 n_days열에 추가
test['n_days'] = (test['Dates'].dt.date - test['Dates'].dt.date.min()).apply(lambda x: x.days) #test데이터 프레임의 tates열에서 날짜값에 가장 작은 날짜값의 차이를 구해서 x.days형태로 n_days열에 추가

**10. 변수 y를 선언해서 학습할 목표변수(=종속변수)인 Category 필드값을 담기**


In [ ]:
y = train['Category'] #Category열을 변수 y로 바인딩

**11. 연도별(year) 범죄 발생 횟수를 알아보기**

**※**value_counts 함수는 데이터 값의 개수를 카운트한다.

**※**sort함수는 데이터 값을 정렬한다. 기본값은 내림차순이고 reverse 옵션을 통해 오름차순도 가능하다.

In [ ]:
pd.value_counts(train['year']).sort_index() #year열의 데이터 값인 년도별로 카운트한 값을 추출

2003    73902
2004    73422
2005    70779
2006    69909
2007    68015
2008    70174
2009    69000
2010    66542
2011    66619
2012    71731
2013    75606
2014    74766
2015    27584
Name: year, dtype: int64

**12. 월별(month) 범죄 발생 횟수를 알아보기**


In [ ]:
pd.value_counts(train['month']).sort_index()  #month열의 데이터 값인 월별로 카운트한 값을 추출

1     73536
2     70813
3     76320
4     78096
5     79644
6     70892
7     69971
8     68540
9     71982
10    80274
11    72975
12    65006
Name: month, dtype: int64

**13. 요일별(DayOfWeek) 범죄 발생 횟수를 알아보기**


In [ ]:
pd.value_counts(train['DayOfWeek']) #DayOfWeek열의 데이터 값인 요일별로 카운트한 값을 추출

Friday       133734
Wednesday    129211
Saturday     126810
Thursday     125038
Tuesday      124965
Monday       121584
Sunday       116707
Name: DayOfWeek, dtype: int64

**14. 카테고리별(Category) 범죄 발생 횟수를 알아보기**


In [ ]:
pd.value_counts(train['Category']) #Category열의 데이터 값인 카테고리별로 카운트한 값을 추출

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

**15. null값이 포함된 데이터 필드가 있는지 확인하기**


In [ ]:
train.isnull().any().sum() #train 데이터프레임셋에 행열구분없이 결측치가 얼마나 있는지 확인

0

**16. 'Dates','Category','Descript','DayOfWeek','Resolution' 컬럼 삭제하기**

**※**drop 함수는 특정 방향의 데이터를 제거한다. axis=0 이면 횡방향 axis=1이면 열방향 데이터를 제거한다.




In [ ]:
train = train.drop(['Dates','Category','Descript','DayOfWeek','Resolution'], axis=1) #'Dates','Category','Descript','DayOfWeek','Resolution'열을 제거